In [ ]:
import socket

# Caesar Cipher Decryption Function
def caesar_decrypt(ciphertext, shift=3):
    decrypted_text = ""
    for char in ciphertext:
        if char.isalpha():
            shift_base = ord('A') if char.isupper() else ord('a')
            decrypted_text += chr((ord(char) - shift_base - shift) % 26 + shift_base)
        else:
            decrypted_text += char  # Keep non-alphabet characters unchanged
    return decrypted_text

# Create UDP server socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

# Dictionary for authenticated users
users = {"user1": "pass1", "user2": "pass2", "admin": "admin123"}
authenticated_clients = {}

print("UDP Chat Server with Encryption is running...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    message = data.decode()

    # Handle authentication
    if client_address not in authenticated_clients:
        try:
            username, password = message.split(",")
        except ValueError:
            server_socket.sendto(b"Invalid login format. Use: username,password", client_address)
            continue

        if username in users and users[username] == password:
            authenticated_clients[client_address] = username
            server_socket.sendto(b"Login successful. You can now chat.", client_address)
            print(f"{username} authenticated from {client_address}")
        else:
            server_socket.sendto(b"Invalid credentials. Try again.", client_address)
        continue

    # Decrypt received message
    decrypted_message = caesar_decrypt(message)
    print(f"{authenticated_clients[client_address]} ({client_address}): {decrypted_message}")

    # Broadcast decrypted message to other authenticated clients
    for client in authenticated_clients:
        if client != client_address:
            encrypted_forward = message  # Forward the encrypted message as-is
            server_socket.sendto(encrypted_forward.encode(), client)
